In [1]:
from firecrawl import FirecrawlApp
import re
from typing import Dict, List, Tuple
app = FirecrawlApp(api_key='fc-406a73525d1c40bdbd9b3d2ad50e99e6')

In [9]:
def get_aliexpress_search_url(q: str):
    q = q.replace(' ', '-')
    url = f"https://pt.aliexpress.com/w/wholesale-{q}.html"
    return url


def parse_links_from_search(response: Dict) -> List[str]:
        
    # Regular expression pattern to match the URLs
    pattern = "pt\.aliexpress\.com/item/\d+\.html"

    # Find all matching URLs in the text
    text = response['markdown']
    matches = re.findall(pattern, text)
    matches = list(set(matches))
    return matches

def parse_product_pictures_urls(product_response) -> List[Dict[str, str]]:
    pattern = "!\[.+?\]\(https://ae-pic-a1\.aliexpress-media\.com/kf/.+?\.jpg_120x120\.jpg_\.webp\)"
    matches = list(set(re.findall(pattern, product_response['markdown'])))
    matches = [m.replace('_120x120.jpg_.webp', '') for m in matches]
    # split product name and url
    output = []
    for m in matches:
        subproduct_name = re.findall('\[(.*?)\]', m)[0]
        subproduct_img_url = re.findall('\((.*?)\)', m)[0]
        output.append({'subproduct_name': subproduct_name, 'subproduct_img_url': subproduct_img_url})
    
    return output






In [3]:
materiais = [
'tecido cetim azul escuro',
'tecido tule preto',
'tecido com paetês dourados',
'tecido veludo azul',
'fita de strass',
'renda preta',
'elastano preto',
'forro azul claro',
]


In [6]:
materiais_data = {}

for m in materiais:
    print('----------------------')
    print('Carregando dados para', m)
    m_data = {}
    m_data['search_url'] = get_aliexpress_search_url(m)
    # for each url, get the links of the products
    m_data['ali_response'] = app.scrape_url(url=m_data['search_url'], params={'formats': [ 'markdown' ]})
    # extract links of products
    m_data['product_urls'] = parse_links_from_search(m_data['ali_response'])
    #products_data = {}
    # for each url, extract the urls of the pictures
    #for url in product_urls:
    #    product_page_response = app.scrape_url(url=url, params={'formats': [ 'markdown' ]})
    #    subproducts_data = parse_product_pictures_urls(product_page_response)
    #    products_data[url] = subproducts_data
    
    materiais_data[m] = m_data
        


----------------------
Carregando dados para tecido cetim azul escuro
----------------------
Carregando dados para tecido tule preto
----------------------
Carregando dados para tecido com paetês dourados
----------------------
Carregando dados para tecido veludo azul
----------------------
Carregando dados para fita de strass
----------------------
Carregando dados para renda preta
----------------------
Carregando dados para elastano preto
----------------------
Carregando dados para forro azul claro


In [8]:
materiais_data['tecido cetim azul escuro']

{'search_url': 'https://pt.aliexpress.com/w/wholesale-tecido-cetim-azul-escuro.html',
 'ali_response': {'markdown': 'tecido cetim azul escuro - Compre tecido cetim azul escuro com envio grátis no AliExpress version\n\n[AliExpress](https://www.aliexpress.com)\n\n![](https://ae01.alicdn.com/kf/S6f0463893f4341e49c75f0dc3bb85ffbo/36x36.png)\n\nTodas as categorias\n\n**Faça o download do aplicativo AliExpress**\n\nOther/PT/ **USD**\n\nOlá! **Entre / Registrar**\n\n[0\\\\\n\\\\\n**Carrinho**\\\\\n\\\\\n0](//www.aliexpress.com/p/shoppingcart/index.html)\n\nOfertas\n\nPromo\n\nOpções de entrega e serviços\n\nChoiceFrete grátis\n\nLocal de envio\n\nTodos\nEstados Unidos\n\nPreço\n\nUS $-US $OK\n\nOrdenar por:\n\nMelhor resultado\n\nPedidos\n\nPreço\n\nVisualização:\n\ngaleria\n\nlista\n\n[![Lago Azul Mulberry Tecido De Seda Chiffon, Lenço Respirável Luz, Saia Flutuante, 50cm x 140cm, 100%](https://pt.aliexpress.com/ae-pic-a1.aliexpress-media.com/kf/S4d7d7ce34e5a44acb9f1c45e7fbdea13i/50cm-140cm-

In [12]:
# iterate over materials_data again to get image urls of subproducts
for material_name, data in materiais_data.items():
    print('Carregando dados para', material_name)
    products_data = {}
    for url in data['product_urls']:
        print("Extracting data for", url)
        product_page_response = app.scrape_url(url=url, params={'formats': [ 'markdown' ]})
        subproducts_data = parse_product_pictures_urls(product_page_response)
        products_data[url] = subproducts_data
    materiais_data[material_name] = products_data
        


Carregando dados para tecido cetim azul escuro
Extracting data for pt.aliexpress.com/item/3256807143248027.html
Extracting data for pt.aliexpress.com/item/3256806983023750.html
Extracting data for pt.aliexpress.com/item/3256802148707345.html
Extracting data for pt.aliexpress.com/item/3256807345408464.html
Extracting data for pt.aliexpress.com/item/3256804696513861.html
Extracting data for pt.aliexpress.com/item/3256807599457010.html
Extracting data for pt.aliexpress.com/item/3256807581802644.html
Extracting data for pt.aliexpress.com/item/3256806156161971.html
Extracting data for pt.aliexpress.com/item/2251832263119749.html
Extracting data for pt.aliexpress.com/item/3256807238705363.html
Extracting data for pt.aliexpress.com/item/3256806622604647.html
Extracting data for pt.aliexpress.com/item/2251832527619073.html
Extracting data for pt.aliexpress.com/item/3256806451703551.html
Extracting data for pt.aliexpress.com/item/3256803220278952.html
Extracting data for pt.aliexpress.com/item/

In [13]:
import json

with open('materiais_scrapped_data_v2.json', 'w') as file:
    json.dump(materiais_data, file)